In [40]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob
import re

nltk.download('punkt')
nltk.download('stopwords')
df=pd.read_excel("Input (copy).xlsx")
df

[nltk_data] Downloading package punkt to /home/harshitha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/harshitha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...
...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...
97,blackassign0098,https://insights.blackcoffer.com/contribution-...
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...


In [41]:
df.columns

Index(['URL_ID', 'URL'], dtype='object')

In [42]:
df.columns

Index(['URL_ID', 'URL'], dtype='object')

In [44]:
def func(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    data = soup.findAll('div', class_="td-post-content tagdiv-type")
    text_file = ""  # Initialize text_file outside the loop
    for d in data:
        d.find_all('p')
        text_file += d.get_text() + " "  # Append to text_file instead of overwriting
        #print(text_file)

    replacements = ['\n', '&', ',', '.', '-', '/', ':', '(', ')', '’', '–', '?']
    for char in replacements:
        text_file = text_file.replace(char, ' ')

    result = []
    text = text_file.split(" ")
    new_file = [item.strip() for item in text if item.strip()]

    stop_words = ["StopWords_Auditor.txt", "StopWords_Currencies.txt", "StopWords_DatesandNumbers.txt", "StopWords_Generic.txt", "StopWords_GenericLong.txt", "StopWords_Geographic.txt", "StopWords_Names.txt"]
    sword_files = []

    for s_w in stop_words:
        with open(s_w, 'r', encoding='iso-8859-1') as file:
            content = file.read().replace('\n',' ').replace('|',' ').split()
            sword_files.extend(content)
    #sword_files

    no_stop_words=[]
    for i in new_file:
        if i not in sword_files:
            no_stop_words.append(i)
    #print(no_stop_words)

    text_file=' '.join(no_stop_words)
    #text_file

    words = nltk.word_tokenize(text_file)
    #print(words)

    positive_words = set(open('positive-words.txt', encoding='ISO-8859-1').read().splitlines())
    negative_words = set(open('negative-words.txt', encoding='ISO-8859-1').read().splitlines())

    filtered_tokens = []

    for token in words:
        token_lower = token.lower()
        if token_lower and token_lower.isalnum() and token_lower not in sword_files:
            filtered_tokens.append(token_lower)

#     with open ("sample.txt",'w+') as file:
#         file.write(text_file)


    positive_score=0
    negative_score = 0
    for token in filtered_tokens:
        if token in positive_words:
            positive_score += 1
        elif token in negative_words:
            negative_score += 1

    # polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    # subjectivity_score = (positive_score + negative_score) / (len(tokens) + 0.000001)

    polarity_score = TextBlob(text_file).polarity
    subjectivity_score = TextBlob(text_file).subjectivity

    # print("positive_score: ", positive_score)
    result.append(positive_score)
    # print("negative_score: ", negative_score)
    result.append(negative_score)
    # print("polarity_score: ", polarity_score)
    result.append(polarity_score)
    # print("subjectivity_score: ", subjectivity_score)
    result.append(subjectivity_score)
    filtered_text=' '.join(filtered_tokens)
    #filtered_text

    import textstat
    avg_sentence_lenght = textstat.avg_sentence_length(filtered_text)
    # print("Average length of the sentence:", avg_sentence_lenght)
    result.append(avg_sentence_lenght)

    def count_syllables(word):
        """Function to count the number of syllables in a word."""
        count = 0
        vowels = 'aeiou'
        word = word.lower()
        if word[0] in vowels:
            count += 1
        for index in range(1, len(word)):
            if word[index] in vowels and word[index - 1] not in vowels:
                count += 1
        if word.endswith('e'):
            count -= 1
        if count == 0:
            count += 1
        return count

    tokens = filtered_text.split()
    total_words = len(tokens)
    complex_word_count = 0
    for token in tokens:
        if count_syllables(token) > 2:
            complex_word_count += 1
#     complex_word_percentage = (complex_word_count / total_words) * 100
#     # print("Complex_word_percentage:" , complex_word_percentage)
#     result.append(complex_word_percentage)

    complex_word_percentage = 0  # Initialize with zero
    if total_words > 0:
        complex_word_percentage = (complex_word_count / total_words) * 100
    # print("Complex_word_percentage:" , complex_word_percentage)
    result.append(complex_word_percentage)


    fog_index = 0.4 * (avg_sentence_lenght + complex_word_percentage)
    # print("Fog_score: ",fog_index)
    result.append(fog_index)

    sentences = filtered_text.split('.')
    #sentences
    total_sentences = len(sentences)
    # print("Total sentences: ", total_sentences)
#     result.append(total_sentences)

    total_words = 0
    for sentence in sentences:
        total_words += len(sentence.split())
    # print("Total Words: ",total_words)
#     result.append(total_words)

    avg_words_per_sentence = total_words / total_sentences
    # print("avg_words_per_sentence: ", avg_words_per_sentence)
    result.append(avg_words_per_sentence)

    # print("complex_word_count: ", complex_word_count)
    result.append(complex_word_count)

    cleaned_word_count = len(no_stop_words)
    # print("cleaned_word_count: ", cleaned_word_count)
    result.append(cleaned_word_count)

    words = filtered_text
    def count_syllablees(word):
        vowels = 'aeiouAEIOU'
        count = 0
        if words.endswith(('es', 'ed')):
            count
        for char in words:
            if char in vowels:
                count += 1
        if words.endswith('e') and words[-2] not in vowels:
            count -= 1
        return max(count, 1)

    syllable_counts = [count_syllablees(word) for word in words]
    total_syllables = sum(syllable_counts)
    # print("total_syllables: ", total_syllables)
    result.append(total_syllables)

#     n=0
#     pronoun_count = filtered_text.split(' ')
#     pronoun = ['i','we','my','ours','us']
#     for i in pronoun_count:
#         if i in pronoun:
#             n+=1
#     # print("Pronoun_count: ",n)


    def count_personal_pronouns(text):
        pronoun_regex = r'\b(?:I|we|my|ours|us)\b'
        pronoun_count = len(re.findall(pronoun_regex, text, re.IGNORECASE))
        return pronoun_count
    
    pronoun_count = count_personal_pronouns(filtered_text)
#   print("Personal pronouns count:", pronoun_count)

    result.append(pronoun_count)

    for word in words:
        total_characters = sum(len(word) for word in filtered_text.split(' '))
    total_words = len(words)
    if total_words > 0:
        average_word_length = total_characters / total_words  
    else:
        average_word_length = 0
        #
    result.append(average_word_length)
#     print(len(result))
    return result
# df['URL'].head(1).apply(func)
print(func("https://insights.blackcoffer.com/how-advertisement-increase-your-market-value/"))

[0, 0, 0.0, 0.0, 0.0, 0, 0.0, 0.0, 0, 0, 0, 0, 0]


In [27]:
rows = df.iloc[:10].copy()
rows[['Positive_Score', 'Negative_Score', 'Polarity_Score',
       'Subjectivity_Score', 'Avg_Sentence_Length', 'Percent_Complex_Words',
       'Fog_Index', 'Avg_Words_per_Sentence', 'Complex_Word_Count',
       'Word_Count', 'Syllable_per_Count', 'Personal_Pronoun',
       'Avg_Word_Length']] = rows['URL'].apply(lambda url: pd.Series(func(url)))
rows

,URL_ID,URL,Positive_Score,Negative_Score,Polarity_Score,Subjectivity_Score,Avg_Sentence_Length,Percent_Complex_Words,Fog_Index,Avg_Words_per_Sentence,Complex_Word_Count,Word_Count,Syllable_per_Count,Personal_Pronoun,Avg_Word_Length
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,34.0,6.0,0.248759,0.586188,548.0,26.277372,229.710949,548.0,144.0,616.0,5653440.0,0.0,0.868510
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,59.0,31.0,0.080707,0.436024,775.0,38.709677,325.483871,775.0,300.0,817.0,14152245.0,0.0,0.880500
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,39.0,26.0,0.077405,0.403071,638.0,48.746082,274.698433,638.0,311.0,686.0,11274752.0,0.0,0.889716
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,36.0,75.0,-0.012171,0.390079,613.0,45.350734,263.340294,613.0,278.0,642.0,10037300.0,0.0,0.889030
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,25.0,9.0,0.169529,0.478227,353.0,36.543909,155.817564,353.0,129.0,381.0,3008838.0,0.0,0.883713
5,blackassign0006,https://insights.blackcoffer.com/the-rise-of-t...,89.0,27.0,0.118954,0.565856,1060.0,44.528302,441.811321,1060.0,472.0,1133.0,30470319.0,0.0,0.888749
6,blackassign0007,https://insights.blackcoffer.com/rise-of-cyber...,32.0,73.0,0.009671,0.455129,696.0,33.764368,291.905747,696.0,235.0,750.0,10112144.0,0.0,0.875738
7,blackassign0008,https://insights.blackcoffer.com/rise-of-inter...,29.0,9.0,0.103136,0.431449,490.0,49.183673,215.673469,490.0,241.0,515.0,6626019.0,0.0,0.888636
8,blackassign0009,https://insights.blackcoffer.com/rise-of-cyber...,38.0,53.0,0.047904,0.408734,602.0,47.009967,259.603987,602.0,283.0,637.0,9946298.0,0.0,0.889907
9,blackassign0010,https://insights.blackcoffer.com/rise-of-cyber...,136.0,187.0,0.073573,0.478412,1774.0,43.010147,726.804059,1774.0,763.0,1950.0,80066448.0,0.0,0.886976


In [45]:
rows = df.iloc[80:100].copy()
rows[['Positive_Score', 'Negative_Score', 'Polarity_Score',
       'Subjectivity_Score', 'Avg_Sentence_Length', 'Percent_Complex_Words',
       'Fog_Index', 'Avg_Words_per_Sentence', 'Complex_Word_Count',
       'Word_Count', 'Syllable_per_Count', 'Personal_Pronoun',
       'Avg_Word_Length']] = rows['URL'].apply(lambda url: pd.Series(func(url)))
rows

,URL_ID,URL,Positive_Score,Negative_Score,Polarity_Score,Subjectivity_Score,Avg_Sentence_Length,Percent_Complex_Words,Fog_Index,Avg_Words_per_Sentence,Complex_Word_Count,Word_Count,Syllable_per_Count,Personal_Pronoun,Avg_Word_Length
80,blackassign0081,https://insights.blackcoffer.com/changing-land...,37.0,49.0,0.023021,0.433443,818.0,32.151589,340.060636,818.0,263.0,891.0,14015430.0,0.0,0.874962
81,blackassign0082,https://insights.blackcoffer.com/online-gaming...,28.0,69.0,-0.026173,0.362740,844.0,37.796209,352.718483,844.0,319.0,884.0,15632994.0,0.0,0.875920
82,blackassign0083,https://insights.blackcoffer.com/human-rights-...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
83,blackassign0084,https://insights.blackcoffer.com/how-voice-sea...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
84,blackassign0085,https://insights.blackcoffer.com/how-the-covid...,39.0,34.0,0.060894,0.416540,733.0,40.791269,309.516508,733.0,299.0,836.0,12053340.0,0.0,0.877325
85,blackassign0086,https://insights.blackcoffer.com/how-to-increa...,57.0,16.0,0.132652,0.400971,868.0,30.645161,359.458065,868.0,266.0,934.0,14847543.0,0.0,0.872293
86,blackassign0087,https://insights.blackcoffer.com/impacts-of-co...,30.0,33.0,0.047570,0.382148,529.0,30.245747,223.698299,529.0,160.0,561.0,5539050.0,0.0,0.871314
87,blackassign0088,https://insights.blackcoffer.com/coronavirus-i...,17.0,44.0,0.016016,0.343507,1014.0,33.037475,418.814990,1014.0,335.0,1080.0,18880299.0,0.0,0.869138
88,blackassign0089,https://insights.blackcoffer.com/coronavirus-i...,17.0,43.0,0.035328,0.442827,483.0,31.262940,205.705176,483.0,151.0,517.0,4613256.0,0.0,0.872115
89,blackassign0090,https://insights.blackcoffer.com/lessons-from-...,35.0,47.0,0.078631,0.415998,514.0,33.073930,218.829572,514.0,170.0,568.0,5666096.0,0.0,0.874695


In [50]:
new_df = pd.DataFrame()
upto = 20
num_rows = 70
for i in range(0, num_rows, upto):
    print("started...")
    end = min(i + upto, num_rows)
    rows = df.iloc[i:end].copy()
    rows[['Positive_Score', 'Negative_Score', 'Polarity_Score',
          'Subjectivity_Score', 'Avg_Sentence_Length', 'Percent_Complex_Words',
          'Fog_Index', 'Avg_Words_per_Sentence', 'Complex_Word_Count',
          'Word_Count', 'Syllable_per_Count', 'Personal_Pronoun',
          'Avg_Word_Length']] = rows['URL'].apply(lambda url: pd.Series(func(url)))
    new_df = pd.concat([new_df, rows], ignore_index=True)  # Update new_df with concatenated result
    print("done....")


started...
done....
started...
done....
started...
done....
started...
done....


In [51]:
len(new_df)

70

In [52]:
new = pd.DataFrame()
upto = 20
num_rows = len(df)
for i in range(70, num_rows, upto):
    print("started...")
    end = min(i + upto, num_rows)
    rows = df.iloc[i:end].copy()
    rows[['Positive_Score', 'Negative_Score', 'Polarity_Score',
          'Subjectivity_Score', 'Avg_Sentence_Length', 'Percent_Complex_Words',
          'Fog_Index', 'Avg_Words_per_Sentence', 'Complex_Word_Count',
          'Word_Count', 'Syllable_per_Count', 'Personal_Pronoun',
          'Avg_Word_Length']] = rows['URL'].apply(lambda url: pd.Series(func(url)))
    new = pd.concat([new, rows], ignore_index=True)  # Update new_df with concatenated result
    print("done....")


started...
done....
started...
done....


In [54]:
result = pd.concat([new,new_df],ignore_index= True)

In [55]:
len(result)

100

In [56]:
result

,URL_ID,URL,Positive_Score,Negative_Score,Polarity_Score,Subjectivity_Score,Avg_Sentence_Length,Percent_Complex_Words,Fog_Index,Avg_Words_per_Sentence,Complex_Word_Count,Word_Count,Syllable_per_Count,Personal_Pronoun,Avg_Word_Length
0,blackassign0071,https://insights.blackcoffer.com/how-to-overco...,25.0,75.0,0.012195,0.621710,404.0,30.693069,173.877228,404.0,124.0,470.0,3293864.0,0.0,0.871779
1,blackassign0072,https://insights.blackcoffer.com/is-perfection...,64.0,22.0,0.357803,0.689069,505.0,24.752475,211.900990,505.0,125.0,556.0,4429152.0,0.0,0.865385
2,blackassign0073,https://insights.blackcoffer.com/global-financ...,27.0,47.0,0.088776,0.454438,634.0,26.182965,264.073186,634.0,166.0,674.0,6697768.0,0.0,0.863812
3,blackassign0074,https://insights.blackcoffer.com/gender-divers...,32.0,27.0,0.074050,0.309192,744.0,34.408602,311.363441,744.0,256.0,807.0,11447460.0,0.0,0.874408
4,blackassign0075,https://insights.blackcoffer.com/how-to-overco...,25.0,84.0,0.000901,0.600317,433.0,30.946882,185.578753,433.0,134.0,501.0,3802498.0,0.0,0.871962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,blackassign0066,https://insights.blackcoffer.com/what-is-the-c...,38.0,26.0,-0.029960,0.353677,694.0,28.530259,289.012104,694.0,198.0,751.0,8948212.0,0.0,0.867723
96,blackassign0067,https://insights.blackcoffer.com/why-does-your...,25.0,7.0,0.225526,0.488306,344.0,37.500000,152.600000,344.0,129.0,399.0,2417415.0,0.0,0.872727
97,blackassign0068,https://insights.blackcoffer.com/how-you-lead-...,41.0,8.0,0.273611,0.415278,525.0,31.047619,222.419048,525.0,163.0,583.0,5673236.0,0.0,0.873185
98,blackassign0069,https://insights.blackcoffer.com/can-you-be-gr...,30.0,6.0,0.239827,0.491522,177.0,41.242938,87.297175,177.0,73.0,196.0,708722.0,0.0,0.876578


In [58]:
new.to_excel('Output2.xlsx',index=False)

In [8]:
import requests
from bs4 import BeautifulSoup


url = "https://insights.blackcoffer.com/rising-it-cities-and-its-impact-on-the-economy-environment-infrastructure-and-city-life-by-the-year-2040-2/"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
data = soup.findAll('div', class_="td-post-content tagdiv-type")
for d in data:
    d.find_all('p')
    text_file=d.get_text()
    #print(text_file)
    
replacements = ['\n', '&', ',', '.', '-', '/', ':', '(', ')', '’', '–', '?']
for char in replacements:
    text_file = text_file.replace(char, ' ')
#text_file

text = text_file.split(" ")
#text

new_file = [item.strip() for item in text if item.strip()]
#new_file

stop_words = ["StopWords_Auditor.txt", "StopWords_Currencies.txt", "StopWords_DatesandNumbers.txt", "StopWords_Generic.txt", "StopWords_GenericLong.txt", "StopWords_Geographic.txt", "StopWords_Names.txt"]
sword_files = []

for s_w in stop_words:
    with open(s_w, 'r', encoding='iso-8859-1') as file:
        content = file.read().replace('\n',' ').replace('|',' ').split()
        sword_files.extend(content)
#sword_files

no_stop_words=[]
for i in new_file:
    if i not in sword_files:
        no_stop_words.append(i)
#print(no_stop_words)

text_file=' '.join(no_stop_words)
#text_file

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


nltk.download('punkt')
nltk.download('stopwords')
words = nltk.word_tokenize(text_file)
#print(words)

positive_words = set(open('positive-words.txt', encoding='ISO-8859-1').read().splitlines())
negative_words = set(open('negative-words.txt', encoding='ISO-8859-1').read().splitlines())

filtered_tokens = []
for token in text_file:
    token_lower = text_file.lower()
    if token_lower.isalnum() and token_lower not in swords_files.words('english'):
        filtered_tokens.append(token_lower)
        
open ("sample.txt",'w+').write(text_file)


with open('sample.txt', 'r') as file:
    content = file.read()
    tokens = content.split()
    
for token in tokens:
    token_lower = token.lower()
    if token_lower.isalnum() and token_lower not in stopwords.words('english'):
        filtered_tokens.append(token_lower)
#print(filtered_tokens)

positive_score=0
negative_score = 0
for token in tokens:
    if token in positive_words:
        positive_score += 1
    elif token in negative_words:
        negative_score += 1
    
polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
subjectivity_score = (positive_score + negative_score) / (len(tokens) + 0.000001)

print("positive_score: ", positive_score)
print("negative_score: ", negative_score)
print("polarity_score: ", polarity_score)
print("subjectivity_score: ", subjectivity_score)

filtered_text=' '.join(filtered_tokens)
#filtered_text

import textstat
avg_sentence_lenght = textstat.avg_sentence_length(filtered_text)
print("Average length of the sentence:", avg_sentence_lenght)

def count_syllables(word):
    """Function to count the number of syllables in a word."""
    count = 0
    vowels = 'aeiouy'
    word = word.lower()
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith('e'):
        count -= 1
    if count == 0:
        count += 1
    return count

tokens = filtered_text.split()
total_words = len(tokens)
complex_word_count = 0
for token in tokens:
    if count_syllables(token) > 2:
        complex_word_count += 1
complex_word_percentage = (complex_word_count / total_words) * 100
print("Complex_word_percentage:" , complex_word_percentage)

fog_index = 0.4 * (avg_sentence_lenght + complex_word_percentage)
print("Fog_score: ",fog_index)

sentences = filtered_text.split('.')
#sentences
total_sentences = len(sentences)
print("Total sentences: ", total_sentences)

total_words = 0
for sentence in sentences:
    total_words += len(sentence.split())
print("Total Words: ",total_words)

avg_words_per_sentence = total_words / total_sentences
print("avg_words_per_sentence: ", avg_words_per_sentence)

print("complex_word_count: ", complex_word_count)

cleaned_word_count = len(no_stop_words)
print("cleaned_word_count: ", cleaned_word_count)

words = filtered_text
def count_syllables(word):
    vowels = 'aeiouAEIOU'
    count = 0
    if words.endswith(('es', 'ed')):
        count
    for char in words:
        if char in vowels:
            count += 1
    if words.endswith('e') and words[-2] not in vowels:
        count -= 1
    return max(count, 1)

syllable_counts = [count_syllables(word) for word in words]
total_syllables = sum(syllable_counts)
print("total_syllables: ", total_syllables)

n=0
pronoun_count = filtered_text.split('.')
pronoun = ['i','we','my','ours','us']
for i in pronoun_count:
    if i in pronoun:
        n+=1
print("Pronoun_count: ",n)

words = filtered_text.split('.')
for word in words:
    total_characters = sum(len(word) for word in words)
total_words = len(words)
if total_words > 0:
    average_word_length = total_characters / total_words  
else:
    average_word_length = 0
print("average_word_length: ", average_word_length)
print(total_characters)



[nltk_data] Downloading package punkt to /home/harshitha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/harshitha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


positive_score:  34
negative_score:  5
polarity_score:  0.7435897245233405
subjectivity_score:  0.0633116882089096
Average length of the sentence: 554.0
Complex_word_percentage: 28.339350180505413
Fog_score:  232.93574007220218
Total sentences:  1
Total Words:  554
avg_words_per_sentence:  554.0
complex_word_count:  157
cleaned_word_count:  616
total_syllables:  5736024
Pronoun_count:  0
average_word_length:  4193.0
4193
